# 导包

In [ ]:
import numpy as np
import pandas as pd

# 细节, 切换下相对路径

In [ ]:
import os

# 手动修改 工作空间目录, 即: 修改相对路径的地址
os.chdir('D:/workspace/ai_20_work_bj/pandasProject/')
os.getcwd()

# 1. apply()函数使用

## 1.1 apply函数作用于 Series对象

In [ ]:
# 1. 创建df对象, 给两列值.
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
df

In [ ]:
# 2. 演示apply()函数, 操作Series对象.
# 需求1: 自定义函数my_func1(), 实现接收Series对象, 然后使其每个值变成 其平方结果.
def my_func1(x):
    return x ** 2


# 传入Series对象, 调用my_func()函数.
# 细节: apply()函数会把 my_func1()函数作用到 Series的每个对象.
df.a.apply(my_func1)  # 细节: 不加小括号, 传入的是函数对象.

In [ ]:
# 需求2: apply传入, 需要多个参数的函数. 例如: 自定义函数my_func2(x, e), 实现计算 x的e次方
def my_func2(x, e):
    return x ** e


# Series调用上述的函数
df['a'].apply(my_func2, e=2)
df['a'].apply(my_func2, e=3)

## 1.2 apply函数作用于 DataFrame对象

In [ ]:
# 细节: apply()函数作用于DF对象, 默认传入的是: axis=0(整列)数据, 不是像Series一样, 逐个元素传递的. 
# 1. 把上述的 my_func1()函数, 作用到DF对象.
df.apply(my_func1)


# 2. 自定义函数my_func3(), 看看df对象到底传入的是什么.
def my_func3(x):
    print(f'x的内容: {x}')
    print(f'x的类型: {type(x)}')


# 3. 调用上述的my_func3(), 作用于: df对象
# df.apply(my_func3)          # 默认传入的就是: 整列(Series对象)
# df.apply(my_func3, axis=0)  # 0:列, 1:行.  该行代码, 效果同上.
df.apply(my_func3, axis=1)  # 传入整行

In [ ]:
# 4. 如下是一种错误示范, 目的是引出稍后要讲解的: 函数的向量化
def avg_3(x, y, z):
    return (x + y + z) / 3


# 无意义, 只是给大家演示下, 如何解决上述的哪个函数出现的问题.
def avg_3_mod(x):
    n1 = x[0]
    n2 = x[1]
    n3 = x[2]
    return (n1 + n2 + n3) / 3


# df.apply(avg_3)         # 报错的, 因为直接传入的是: 整列的数据.
df.apply(avg_3_mod)

## 1.3 apply()函数案例-泰坦尼克号数据集

In [ ]:
# 需求: 自定义函数, 分别计算 泰坦尼克号数据集 某列的缺失值个数, 某列的缺失值占比, 某列的非缺失值占比.
# 1. 读取数据源, 获取df对象.
titanic = pd.read_csv('data/titanic_train.csv')
titanic.head()

In [ ]:
# 2. 定义函数, 实现各种需求.
# 需求1: count_missing(vec), 计算某列的缺失值个数
def count_missing(vec):
    # vec就是接收到的 df对象的 某列 或者 某行数据
    return pd.isnull(vec).sum()


# 需求2: prop_missing(vec), 计算某列中缺失值占比
def prop_missing(vec):
    # 缺失值占比公式: 某列缺失值数量 / 某列的元素总个数
    # return pd.isnull(vec).sum() / vec.size
    return count_missing(vec) / vec.size


# 需求3: prop_complete(vec), 计算某列的 非缺失值占比.  # vector
def prop_complete(vec):
    # 非缺失值占比: 1 - 缺失值占比
    return 1 - prop_missing(vec)


In [ ]:
# 3. 测试上述的函数.
# 默认: axis = 0, 即: 以列的形式传入的
titanic.apply(count_missing)  # 传入的时候一定不要加小括号, 我们传入的是: 函数对象
titanic.apply(prop_missing)  # 某列缺失值占比
titanic.apply(prop_complete)  # 某列非缺失值占比

# 以行的形式传入
titanic.apply(count_missing, axis=1)  # 1代表: 行, 某行缺失值数量
titanic.apply(prop_missing, axis=1)  # 某行缺失值占比
titanic.apply(prop_complete, axis=1)  # 某行非缺失值占比

titanic.shape  # 维度(行列数), (891, 12)

## 1.4 向量化函数介绍, np.vectorize

In [ ]:
# 1. 创建df对象, 给两列值.
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
df

In [ ]:
# 2. 定义函数 avg_2, 计算上述的平均值.
def avg_2(x, y):
    return (x + y) / 2


# 调用上述的自定义函数
avg_2(df['a'], df['b'])  # 无问题

In [ ]:
# 3. 改造上述的代码, 程序出问题了.
def avg_2_mod(x, y):
    if x == 20:  # 这里会出错, 因为: x是向量(简单理解: 一堆值), 20是标量(1个值)
        return np.NaN
    else:
        return (x + y) / 2


# 调用上述的自定义函数
# avg_2_mod(df['a'], df['b'])     # 有问题

# 解决思路: 通过np.vectorize将上述的函数转成: 向量化函数, 如果函数中遇到向量了, 则内部会自动遍历.
# 写法1: np.vectorize修饰函数, 获取: 向量化后的函数.
avg_2_mod_vec = np.vectorize(avg_2_mod)
avg_2_mod_vec(df['a'], df['b'])

In [ ]:
# 写法2: 装饰器方式, 装饰 函数.
@np.vectorize
def avg_2_mod(x, y):
    if x == 20:  # 这里会出错, 因为: x是向量(简单理解: 一堆值), 20是标量(1个值)
        return np.NaN
    else:
        return (x + y) / 2


# 调用上述的自定义函数
avg_2_mod(df['a'], df['b'])

## 1.5 apply()函数, 接收 lambda表达式写的 匿名函数

In [ ]:
# 细节: 如果需求非常简单, 我们就没有必要再去定义函数了, 直接传入 lambda表达式即可.
# 需求: df的每个元素, 变成其平方值.
# 1. 创建df对象, 给两列值.
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
df

In [ ]:
# 方式1: 普通写法.
def my_fun1(x):
    return x ** 2


# 调用函数
df.apply(my_fun1)

In [ ]:
# 方式2: lambda 表达式
df.apply(lambda x: x ** 2)
df.apply(lambda x: x ** 3)
df.apply(lambda x: x + 1)

# 2. 数据分组处理

## 2.1 分组聚合,  n => 1 

In [ ]:
# 格式: groupby([分组字段1, 分组字段2...])[[要聚合运算的字段1, 字段2...]].聚合函数名()
# 1. 加载数据源, 获取df对象
df = pd.read_csv('data/gapminder.tsv', sep='\t')
df.head()

In [ ]:
# 2. 演示 分组聚合 操作, 即: 分完组, 聚合计算后, 每组都只会获取 1个 结果.
# 需求1: 统计每年的 平均寿命.
df.groupby('year').lifeExp.mean()

# 需求2: 针对于上述的结果, 我们还可以手动, 逐个计算每年的平均寿命.
# 1. 看看有哪些年.
df['year'].unique()  # 一共有 12个 年份, 从1952年开始统计的. 

# 2. 手动计算第1组, 即: 1952年的平均寿命
df[df.year == 1952].lifeExp.mean()  # 49.05761971830987

# 3. 其实上述的分组, 内部就是逐个的计算每年的平均寿命, 然后组合到一起.

In [ ]:
# 需求3: 上述我们使用的是pandas的聚合函数, 我们还可以使用 Numpy的聚合函数
# 例如: 统计各个大洲的 平均寿命.
df.groupby('continent').lifeExp.mean()  # mean: pandas的函数
df.groupby('continent').lifeExp.agg(np.mean)  # 效果同上, np.mean 是 numpy的函数

df.groupby('continent').agg({'lifeExp': 'mean'})  # pandas的聚合函数
df.groupby('continent').agg({'lifeExp': np.mean})  # numpy的聚合函数

# 使用aggregate() 和 agg()函数效果是一样的.
df.groupby('continent').aggregate({'lifeExp': 'mean'})  # pandas的聚合函数
df.groupby('continent').aggregate({'lifeExp': np.mean})  # numpy的聚合函数

In [ ]:
# 需求4: 使用自定义的函数, 完成: 计算平均值.
def my_mean(col):
    # return col.mean()
    return col.sum() / col.size  # 总和 / 个数

# 传入我们自定义的函数, 计算平均值
df.groupby('continent').lifeExp.agg(my_mean)  # 不要加小括号, 这里传入的是: 函数对象.

In [ ]:
# 需求5: 计算全球平均预期寿命的平均值 和 分组之后的平均值做差值.
# 大白话: 就是上述算出来的各组平均值 和 全球总预期寿命平均值的差值.
def my_mean_diff(col, diff_value):
    # return my_mean(col) - diff_value
    return col.mean() - diff_value


# 1. 获取全球平均预期寿命.
global_lifeExp_mean = df.lifeExp.mean()

# 2. 完成上述的需求. 
df.groupby('continent').lifeExp.agg(my_mean_diff, diff_value=global_lifeExp_mean)

In [ ]:
# 需求6: agg()函数, 同时传入多个函数.
# 例如: 按年计算LifeExp的 非零个数, 平均值 和 标准差.
df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])

# 例如: 计算多列值的不同需求, 按年统计, 平均寿命, 最大人口, 最小GDP
df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'max', 'gdpPercap': 'min'})

## 2.2 分组转换.    n => n

In [ ]:
# 1. 查看数据源. 
df.head()

In [ ]:
# 2. 需求1: 自定义函数, 计算某列的 Z分数(标准分数).  公式: (x - x的平均值) / 标准差
def my_zscore(x):
    return (x - x.mean()) / x.std()

# 3. 按年分组, 计算 LifeExp列的, zscore标准分数.
df.groupby('year').lifeExp.transform(my_zscore)     # 1704
# df.shape                                          # 维度: 行列数, (1704, 6)

In [ ]:
# 需求2: 按照性别分组, 计算男女(male, female)各组的平均消费, 然后进行填充.
# 1. 读取数据源, 获取df对象.
# sample(): 随机取样的, 参1: 取几个值.  参2: 随机种子, 如果种子一样, 每次获取的结果都是一样的. 
tips_10 = pd.read_csv('data/tips.csv').sample(10, random_state=20)
tips_10

In [ ]:
# 2. 人为的给 total_bill列, 创建几个空值.
# tips_10.loc[[87, 153, 128], 'total_bill'] = np.NaN       # 可以实现需求, 但是不好, 空值我们都固定了.
# np.random.permutation(tips_10.index)  根据 tips_10 这个df对象的索引, 对其进行随机打散
tips_10.loc[np.random.permutation(tips_10.index)[:4] , 'total_bill'] = np.NaN

# 3. 查看各列的非空值统计情况.
tips_10

In [ ]:
# 4. 我不想用 total_bill列的平均值, 来填充该列的空值, 因为: 男, 女一般消费习惯不一样.
# 我想要做的事情是, 计算出 男, 女各自的 平均总消费, 然后按照性别来填充. 
# 自定义函数, 用于实现, 用该列的平均值, 来填充数据.
def my_fillna(col):
    return col.fillna(col.mean())

# 调用上述的函数, 填充即可.  transform(): 分组转换, 即: 源数据无论多少条, 处理后还是多少条, 类似于SQL的窗口函数.
tips_10.groupby('sex').total_bill.transform(my_fillna)

## 2.3 分组过滤

In [ ]:
# 1. 加载数据源.
tips = pd.read_csv('data/tips.csv')
tips

# 2. 查看吃饭人数的分布情况
tips['size'].value_counts()

# 3. 发现, 1个人, 5个人, 6个人吃饭的次数还是较少的, 我们过滤掉这部分数据. 
tmp_df = tips.groupby('size').filter(lambda x: x['size'].count() > 10)

# 4. 过滤完后, 查看过滤后的结果.
tmp_df['size'].value_counts()

## 2.4 分组对象 DataFrameGroupBy演示 

In [ ]:
# 1. 加载数据源, 获取df对象.
df = pd.read_csv('data/tips.csv').sample(10, random_state=20)   # 随机种子一致, 每次抽取的结果都是一样的
df

In [ ]:
# 2. 根据性别分组, 获取分组对象.
grouped = df.groupby('sex')       # DataFrameGroupBy 数据类型

# 3. 通过 groups 属性查看计算过的分组. 
grouped.groups      # {'Female': [209, 128, 132], 'Male': [87, 236, 153, 172, 237, 129, 13]}

# 4. 在DataFrameGroupBy基础上, 可以直接计算.
grouped.mean()      # 等价于: df.groupby('sex').mean() 

# 5. 通过 get_group 获取分组.
# grouped[0]                # 错误写法
grouped.get_group('Male')   # 正确写法............

# 6. 遍历groupby对象, 获取所有的分组
for sex_group in grouped:
    print(sex_group)        # 查看分组对象, 各组的数据.
    
# 7. 需求: 按性别和用餐时间分组, 计算小费数据的平均值.
df.groupby(['sex', 'time']).tip.mean()
df.groupby(['sex', 'time'], as_index=False).tip.mean()  # as_index=False 不把分组字段作为 索引列